In [1]:
import pandas as pd
import os


FileNotFoundError: ignored

In [2]:
sentiment_labels = pd.read_csv(os.path.join("/content/sentiment_labels.txt"), sep="|")
sentence_ids = pd.read_csv(os.path.join("/content/datasetSentences.txt"), sep="\t")
dictionary = pd.read_csv(os.path.join("/content/dictionary.txt"), sep="|", names=['phrase', 'phrase ids'])
train_test_split = pd.read_csv(os.path.join("/content/datasetSplit.txt"))
sentence_phrase_merge = pd.merge(sentence_ids, dictionary, left_on='sentence', right_on='phrase')
sentence_phrase_split = pd.merge(sentence_phrase_merge, train_test_split, on='sentence_index')
df = pd.merge(sentence_phrase_split, sentiment_labels, on='phrase ids').sample(frac=1)

In [3]:
df.shape

(11214, 6)

In [4]:
df.head(10)

,sentence_index,sentence,phrase,phrase ids,splitset_label,sentiment values
9314,9820,Wobbly Senegalese updating of `` Carmen '' whi...,Wobbly Senegalese updating of `` Carmen '' whi...,227208.0,1,0.59722
326,335,"Finally , a genre movie that delivers -- in a ...","Finally , a genre movie that delivers -- in a ...",18871.0,2,0.79167
1136,1188,It 's refreshing to see a girl-power movie tha...,It 's refreshing to see a girl-power movie tha...,25763.0,3,0.84722
662,694,Good movie .,Good movie .,40921.0,2,0.90278
6645,6994,The movie is a desperate miscalculation .,The movie is a desperate miscalculation .,149592.0,1,0.16667
3502,3688,More good than great but Freeman and Judd make...,More good than great but Freeman and Judd make...,67610.0,1,0.70833
2603,2740,"Talk to Her is so darned assured , we have abs...","Talk to Her is so darned assured , we have abs...",69146.0,1,0.69444
7377,7777,Let 's hope -- shall we ?,Let 's hope -- shall we ?,224604.0,3,0.50000
2570,2704,A compelling pre-WWII drama with vivid charact...,A compelling pre-WWII drama with vivid charact...,62873.0,1,0.84722
6221,6551,"Although bright , well-acted and thought-provo...","Although bright , well-acted and thought-provo...",104186.0,1,0.44444


In [5]:
df.splitset_label.value_counts()

1    8066
2    2112
3    1036
Name: splitset_label, dtype: int64

In [6]:
def discretize_label(label):
    if label <= 0.2: return 'Class1'
    if label <= 0.4: return 'Class2'
    if label <= 0.6: return 'Class3'
    if label <= 0.8: return 'Class4'
    return 'Class5'

In [7]:
import random
import torch, torchtext
from torchtext import data

In [8]:
# Manual Seed
SEED = 43
torch.manual_seed(SEED)

In [9]:
Tweet = torchtext.legacy.data.Field(sequential = True, tokenize = 'spacy', batch_first =True, include_lengths=True)
Label = torchtext.legacy.data.LabelField(tokenize ='spacy', is_target=True, batch_first =True, sequential =False)

In [10]:
fields = [('phrase', Tweet),('splitset_label',Label)]

In [13]:
example = [torchtext.legacy.data.Example.fromlist([df.phrase[i],df.splitset_label[i]], fields) for i in range(df.shape[0])] 

In [16]:
twitterDataset = torchtext.legacy.data.Dataset(example, fields)

In [17]:
(train, valid) = twitterDataset.split(split_ratio=[85, 15], random_state = random.seed(SEED))

In [18]:
len(train), len(valid)

(9532, 1682)

In [19]:
vars(train.examples[11])

{'phrase': ['It',
  "'s",
  'one',
  'of',
  'the',
  'saddest',
  'films',
  'I',
  'have',
  'ever',
  'seen',
  'that',
  'still',
  'manages',
  'to',
  'be',
  'uplifting',
  'but',
  'not',
  'overly',
  'sentimental',
  '.'],
 'splitset_label': 2}

In [20]:
Tweet.build_vocab(train)
Label.build_vocab(train)

In [21]:
print('Size of input vocab : ', len(Tweet.vocab))
print('Size of label vocab : ', len(Label.vocab))
print('Top 10 words appreared repeatedly :', list(Tweet.vocab.freqs.most_common(10)))
print('Labels : ', Label.vocab.stoi)

Size of input vocab :  18010
Size of label vocab :  3
Top 10 words appreared repeatedly : [('.', 8985), (',', 8010), ('the', 6728), ('and', 4894), ('a', 4886), ('of', 4883), ('to', 3388), ('-', 3040), ('is', 2849), ("'s", 2812)]
Labels :  defaultdict(None, {1: 0, 2: 1, 3: 2})


In [23]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [24]:
train_iterator, valid_iterator = torchtext.legacy.data.BucketIterator.splits((train, valid), batch_size = 32, 
                                                            sort_key = lambda x: len(x.phrase),
                                                            sort_within_batch=True, device = device)

In [25]:
next(iter(train_iterator))
#len(train.examples[11].tweet)


[torchtext.legacy.data.batch.Batch of size 32]
	[.phrase]:('[torch.cuda.LongTensor of size 32x9 (GPU 0)]', '[torch.cuda.LongTensor of size 32 (GPU 0)]')
	[.splitset_label]:[torch.cuda.LongTensor of size 32 (GPU 0)]

In [26]:
import os, pickle
with open('tokenizer.pkl', 'wb') as tokens: 
    pickle.dump(Tweet.vocab.stoi, tokens)

In [27]:
import torch.nn as nn
import torch.nn.functional as F

class classifier(nn.Module):
    
    # Define all the layers used in model
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, dropout):
        
        super().__init__()          
        
        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        # LSTM layer
        self.encoder = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           dropout=dropout,
                           batch_first=True)
        # try using nn.GRU or nn.RNN here and compare their performances
        # try bidirectional and compare their performances
        
        # Dense layer
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text, text_lengths):
        
        # text = [batch size, sent_length]
        embedded = self.embedding(text)
        # embedded = [batch size, sent_len, emb dim]
      
        # packed sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.cpu(), batch_first=True)
        
        packed_output, (hidden, cell) = self.encoder(packed_embedded)
        #hidden = [batch size, num layers * num directions,hid dim]
        #cell = [batch size, num layers * num directions,hid dim]
    
        # Hidden = [batch size, hid dim * num directions]
        dense_outputs = self.fc(hidden)   
        
        # Final activation function softmax
        output = F.softmax(dense_outputs[0], dim=1)
            
        return output

In [28]:
# Define hyperparameters
size_of_vocab = len(Tweet.vocab)
embedding_dim = 300
num_hidden_nodes = 100
num_output_nodes = 3
num_layers = 2
dropout = 0.2

# Instantiate the model
model = classifier(size_of_vocab, embedding_dim, num_hidden_nodes, num_output_nodes, num_layers, dropout = dropout)

In [29]:
print(model)

#No. of trianable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
print(f'The model has {count_parameters(model):,} trainable parameters')

classifier(
  (embedding): Embedding(18010, 300)
  (encoder): LSTM(300, 100, num_layers=2, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=100, out_features=3, bias=True)
)
The model has 5,644,903 trainable parameters


In [38]:
import torch.optim as optim

# define optimizer and loss
optimizer = optim.Adam(model.parameters(), lr=2e-4)
criterion = nn.CrossEntropyLoss()

# define metric
def binary_accuracy(preds, y):
    #round predictions to the closest integer
    _, predictions = torch.max(preds, 1)
    
    correct = (predictions == y).float() 
    acc = correct.sum() / len(correct)
    return acc
    
# push to cuda if available
model = model.to(device)
criterion = criterion.to(device)

In [44]:
def train(model, iterator, optimizer, criterion):
    
    # initialize every epoch 
    epoch_loss = 0
    epoch_acc = 0
    
    # set the model in training phase
    model.train()  
    
    for batch in iterator:
        
        # resets the gradients after every batch
        optimizer.zero_grad()   
        
        # retrieve text and no. of words
        tweet, tweet_lengths = batch.phrase
        
        # convert to 1D tensor
        predictions = model(tweet, tweet_lengths).squeeze()  
        
        # compute the loss
        loss = criterion(predictions, batch.splitset_label)        
        
        # compute the binary accuracy
        acc = binary_accuracy(predictions, batch.splitset_label)   
        
        # backpropage the loss and compute the gradients
        loss.backward()       
        
        # update the weights
        optimizer.step()      
        
        # loss and accuracy
        epoch_loss += loss.item()  
        epoch_acc += acc.item()    
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [45]:
def evaluate(model, iterator, criterion):
    
    # initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    # deactivating dropout layers
    model.eval()
    
    # deactivates autograd
    with torch.no_grad():
    
        for batch in iterator:
        
            # retrieve text and no. of words
            tweet, tweet_lengths = batch.phrase
            
            # convert to 1d tensor
            predictions = model(tweet, tweet_lengths).squeeze()
            
            # compute loss and accuracy
            loss = criterion(predictions, batch.splitset_label)
            acc = binary_accuracy(predictions, batch.splitset_label)
            
            # keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [46]:
N_EPOCHS = 10
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
     
    # train the model
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    
    # evaluate the model
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    # save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}% \n')

	Train Loss: 0.832 | Train Acc: 71.98%
	 Val. Loss: 0.836 |  Val. Acc: 71.59% 

	Train Loss: 0.831 | Train Acc: 72.01%
	 Val. Loss: 0.837 |  Val. Acc: 71.59% 

	Train Loss: 0.830 | Train Acc: 72.13%
	 Val. Loss: 0.837 |  Val. Acc: 71.53% 

	Train Loss: 0.829 | Train Acc: 72.29%
	 Val. Loss: 0.838 |  Val. Acc: 71.29% 

	Train Loss: 0.829 | Train Acc: 72.30%
	 Val. Loss: 0.838 |  Val. Acc: 71.41% 

	Train Loss: 0.828 | Train Acc: 72.38%
	 Val. Loss: 0.838 |  Val. Acc: 71.35% 

	Train Loss: 0.827 | Train Acc: 72.46%
	 Val. Loss: 0.839 |  Val. Acc: 71.35% 

	Train Loss: 0.827 | Train Acc: 72.48%
	 Val. Loss: 0.838 |  Val. Acc: 71.41% 

	Train Loss: 0.827 | Train Acc: 72.51%
	 Val. Loss: 0.838 |  Val. Acc: 71.41% 

	Train Loss: 0.826 | Train Acc: 72.55%
	 Val. Loss: 0.839 |  Val. Acc: 71.35% 



In [50]:
#load weights and tokenizer

path='./saved_weights.pt'
model.load_state_dict(torch.load(path));
model.eval();
tokenizer_file = open('./tokenizer.pkl', 'rb')
tokenizer = pickle.load(tokenizer_file)

#inference 

import spacy
nlp = spacy.load('en')

def classify_tweet(phrase):
    
    categories = {0: "Negative", 1:"Positive", 2:"Neutral"}
    
    # tokenize the tweet 
    tokenized = [tok.text for tok in nlp.tokenizer(phrase)] 
    # convert to integer sequence using predefined tokenizer dictionary
    indexed = [tokenizer[t] for t in tokenized]        
    # compute no. of words        
    length = [len(indexed)]
    # convert to tensor                                    
    tensor = torch.LongTensor(indexed).to(device)   
    # reshape in form of batch, no. of words           
    tensor = tensor.unsqueeze(1).T  
    # convert to tensor                          
    length_tensor = torch.LongTensor(length)
    # Get the model prediction                  
    prediction = model(tensor, length_tensor)

    _, pred = torch.max(prediction, 1) 
    
    return categories[pred.item()]

In [51]:
classify_tweet("A valid explanation for why Trump won't let women on the golf course.")

'Negative'